In [46]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('dataset/tripdata.csv')
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,41.915689,-87.634600,member
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,41.864221,-87.623439,casual
2,67DC1D133E8B5816,electric_bike,2020-08-26 19:44:14,2020-08-26 21:53:07,Columbus Dr & Randolph St,195.0,State St & Randolph St,44.0,41.884641,-87.619549,41.884971,-87.627574,casual
3,C79FBBD412E578A7,electric_bike,2020-08-27 12:05:41,2020-08-27 12:53:45,Daley Center Plaza,81.0,State St & Kinzie St,47.0,41.884093,-87.629639,41.889583,-87.627540,casual
4,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,41.903002,-87.683844,casual


In [47]:
# Drop the columns that are not needed
df = df.drop(["rideable_type", "member_casual"], axis=1)

# Convert the started_at and ended_at columns to datetime
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

# Calculate the ride duration in minutes
df['ride_duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60
# Remove the rows with negative ride duration
df = df[df['ride_duration'] > 0].copy()

df.head()

,ride_id,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,ride_duration
0,322BD23D287743ED,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,41.915689,-87.634600,9.616667
1,2A3AEF1AB9054D8B,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,41.864221,-87.623439,68.783333
2,67DC1D133E8B5816,2020-08-26 19:44:14,2020-08-26 21:53:07,Columbus Dr & Randolph St,195.0,State St & Randolph St,44.0,41.884641,-87.619549,41.884971,-87.627574,128.883333
3,C79FBBD412E578A7,2020-08-27 12:05:41,2020-08-27 12:53:45,Daley Center Plaza,81.0,State St & Kinzie St,47.0,41.884093,-87.629639,41.889583,-87.627540,48.066667
4,13814D3D661ECADB,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,41.903002,-87.683844,10.783333


In [48]:
# Count rows with empty start_station_id
print(f"Number of rows with empty start_station_id: {df['start_station_id'].isna().sum()}")
# Drop rows with empty start_station_id
df = df.dropna(subset=['start_station_id'])

print(f"Number of rows with empty start_station_id: {df['start_station_id'].isna().sum()}")

Number of rows with empty start_station_id: 7691
Number of rows with empty start_station_id: 0


In [49]:
# Count rows with empty end_station_id
print(f"Number of rows with empty end_station_id: {df['end_station_id'].isna().sum()}")
# Drop rows with empty start_station_id
df = df.dropna(subset=['end_station_id'])

print(f"Number of rows with empty end_station_id: {df['end_station_id'].isna().sum()}")

Number of rows with empty end_station_id: 6138
Number of rows with empty end_station_id: 0


In [50]:
# Count rows with empty values
df.isnull().sum().sum()
# Prints number of rows of the cleaned dataset
print(len(df))

605652


In [51]:
# Count the number of unique start stations
print(f"Real number of start stations:", len(df['start_station_id'].unique()))

Real number of start stations: 633


In [52]:
# Group by start_station_id and calculate the mean for start_lat and end_lat
start_station_map = df.groupby('start_station_id')[['start_lat', 'start_lng']].mean().reset_index()

# Rename the column to 'station_id'
start_station_map.rename(columns={'start_station_id': 'station_id', 'start_lat': 'lat', 'start_lng': 'lng'}, inplace=True)

# Group by start_station_id and calculate the mean for start_lat and end_lat
end_station_map = df.groupby('end_station_id')[['end_lat', 'end_lng']].mean().reset_index()

# Rename the column to 'station_id'
end_station_map.rename(columns={'end_station_id': 'station_id','end_lat': 'lat', 'end_lng': 'lng'}, inplace=True)

# Merge end_station_map with start_station_map on station_id
# Merge end_station_map with start_station_map on station_id and calculate the mean of lat and lng values
stations_map = pd.merge(start_station_map, end_station_map, on='station_id', suffixes=('_start', '_end'))

# Calculate the mean of lat and lng values
stations_map['lat'] = stations_map[['lat_start', 'lat_end']].mean(axis=1)
stations_map['lng'] = stations_map[['lng_start', 'lng_end']].mean(axis=1)

# Drop the intermediate columns
stations_map = stations_map.drop(columns=['lat_start', 'lat_end', 'lng_start', 'lng_end'])

# Display the resulting dataframe
stations_map.head()

,station_id,lat,lng
0,2.0,41.876508,-87.620542
1,3.0,41.867200,-87.615353
2,4.0,41.856258,-87.613339
3,5.0,41.874052,-87.627711
4,6.0,41.886972,-87.612814


In [53]:
# Extract unique start station names and their corresponding IDs
start_stations = df[['start_station_name', 'start_station_id']].drop_duplicates().reset_index(drop=True)

# Extract unique end station names and their corresponding IDs
end_stations = df[['end_station_name', 'end_station_id']].drop_duplicates().reset_index(drop=True)

# Rename columns to have consistent names for concatenation
end_stations.columns = ['station_name', 'station_id']
start_stations.columns = ['station_name', 'station_id']

# Concatenate start and end stations
all_stations_names = pd.concat([start_stations, end_stations]).drop_duplicates().reset_index(drop=True)

# Display the resulting dataframe
all_stations_names.head()

,station_name,station_id
0,Lake Shore Dr & Diversey Pkwy,329.0
1,Michigan Ave & 14th St,168.0
2,Columbus Dr & Randolph St,195.0
3,Daley Center Plaza,81.0
4,Leavitt St & Division St,658.0


In [54]:
# Merge all_stations_names with stations_map on station_id
all_stations = pd.merge(all_stations_names, stations_map, on='station_id')

# Reorder columns to put station_id first
all_stations = all_stations[['station_id', 'station_name', 'lat', 'lng']]

# Order rows by ascending order of station_id
all_stations = all_stations.sort_values(by='station_id').reset_index(drop=True)

# Display the resulting dataframe
all_stations.head()

all_stations.to_csv('./cleaned_datasets/all_stations.csv', index=False)

In [55]:
# Haversine formula function to calculate distance between two lat/long points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c  # Distance in kilometers
    return distance

In [56]:
all_trips = df.drop(["start_station_name", "end_station_name", "start_lat", "start_lng", "end_lat", "end_lng"], axis=1)
all_trips['distance'] = 0
#all_trips.head()

#for i, row in all_trips.iterrows():

#    start_lat = all_stations[all_stations['station_id'] == row['start_station_id']]["lat"]
#    start_lng = all_stations[all_stations['station_id'] == row['start_station_id']]["lng"]

#    end_lat = all_stations[all_stations['station_id'] == row['end_station_id']]["lat"]
#    end_lng = all_stations[all_stations['station_id'] == row['end_station_id']]["lng"]

#    distance = haversine(start_lat, start_lng, end_lat, end_lng)
#    row['distance'] = distance

#all_trips.head()

all_trips.to_csv('./cleaned_datasets/all_trips.csv', index=False)